In [1]:
import pandas as pd

In [2]:
training_data= pd.read_csv("X_Y_train.csv",delimiter = ",", encoding = 'latin-1')
testing_data= pd.read_csv("X_test.csv",delimiter = ",", encoding = 'latin-1')


In [3]:
training_data.shape

(10980, 12)

In [4]:
testing_data.head()

,tweet_id,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,569682010270101504,American,NaN,zsalim03,NaN,0,@AmericanAir In car gng to DFW. Pulled over 1h...,NaN,2015-02-22 18:15:50 -0800,Texas,Central Time (US & Canada)
1,569608307184242688,American,NaN,sa_craig,NaN,0,"@AmericanAir after all, the plane didnât lan...",NaN,2015-02-22 13:22:57 -0800,"College Station, TX",Central Time (US & Canada)
2,567879304593408001,Southwest,NaN,DanaChristos,NaN,1,@SouthwestAir can't believe how many paying cu...,NaN,2015-02-17 18:52:31 -0800,CT,Eastern Time (US & Canada)
3,569757651539660801,US Airways,NaN,rossj987,NaN,0,@USAirways I can legitimately say that I would...,NaN,2015-02-22 23:16:24 -0800,"Washington, D.C.",Eastern Time (US & Canada)
4,569900705852608513,American,NaN,tranpham18,NaN,0,@AmericanAir still no response from AA. great ...,NaN,2015-02-23 08:44:51 -0800,New York City,Eastern Time (US & Canada)


In [5]:
X= training_data.iloc[:,7]
Y= training_data.iloc[:,1]
x_test= testing_data.iloc[:,6]

In [6]:
def y_transform(y):
    if(y=="positive"):
        return 0
    elif(y=="neutral"):
        return 1
    else:
        return 2

In [7]:
Y=Y.apply(y_transform)

In [8]:
from nltk.tokenize import word_tokenize

In [9]:
def forming_words(string):
    return word_tokenize(string)

In [10]:
X= X.apply(forming_words)
x_test= x_test.apply(forming_words)

In [11]:
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
leammatizer= WordNetLemmatizer()

In [12]:
from nltk.corpus import stopwords
import string

stops= set(stopwords.words('english'))
punctuations= list(string.punctuation)
stops.update(punctuations)

In [13]:
from nltk.corpus import wordnet
def get_simple_pos(tag):
    
    if(tag.startswith('J')):
        return wordnet.ADJ
    elif(tag.startswith('V')):
        return wordnet.VERB
    elif(tag.startswith('N')):
        return wordnet.NOUN
    elif(tag.startswith('R')):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [14]:
def clean_review(words):
    output_words=[]
    
    for w in words:
        if(w.lower() not in stops):
            pos= pos_tag([w])
            clean_word = leammatizer.lemmatize(w,pos=get_simple_pos(pos[0][1]))
            output_words.append(clean_word.lower())
    return output_words

In [15]:
X= X.apply(clean_review)
x_test= x_test.apply(clean_review)

In [16]:
def merging_words(words):
    return " ".join(words)

In [17]:
X= X.apply(merging_words)
x_test= x_test.apply(merging_words)

In [18]:
from sklearn.feature_extraction.text import CountVectorizer

In [19]:
count_vec= CountVectorizer(max_features=8370,ngram_range=(1,2))

In [20]:
x_train_features = count_vec.fit_transform(X)

In [21]:
x_train_features.todense()

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [22]:
Y.shape

(10980,)

In [23]:
x_test_features= count_vec.transform(x_test)

In [24]:
x_test_features.todense()

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [25]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x_train_features, Y, test_size = 0.2, random_state = 101)

In [26]:
from sklearn.naive_bayes import MultinomialNB
clf= MultinomialNB()
clf.fit(X_train,y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [27]:
y_pred = clf.predict(X_test)
y_pred

array([2, 2, 2, ..., 2, 1, 1], dtype=int64)

In [28]:
y_test=y_test.to_numpy()

In [29]:
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.71      0.66      0.68       372
           1       0.59      0.53      0.56       451
           2       0.84      0.88      0.86      1373

    accuracy                           0.77      2196
   macro avg       0.71      0.69      0.70      2196
weighted avg       0.77      0.77      0.77      2196



In [30]:
print(confusion_matrix(y_test,y_pred))

[[ 244   59   69]
 [  48  237  166]
 [  53  105 1215]]


In [34]:
from sklearn.linear_model import LogisticRegression
clf= LogisticRegression()
clf.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [37]:
y_pred = clf.predict(X_test)

In [36]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.77      0.61      0.68       372
           1       0.63      0.59      0.61       451
           2       0.84      0.91      0.87      1373

    accuracy                           0.79      2196
   macro avg       0.75      0.70      0.72      2196
weighted avg       0.79      0.79      0.79      2196

